In [1]:
import base64
import email
import os
import re

from lxml import etree

In [2]:
def parse_old_mail(content: str) -> dict | None:
    tree = etree.HTML(content)
    lessons = tree.xpath(
        "//*[contains(text(),'Lessons finished')]/ancestor::tr[1]/following-sibling::tr[1]//b[@class='card-value chameleon-heading']/text()"
    )
    minutes = tree.xpath(
        "//*[contains(text(),'Time learning')]/ancestor::tr[1]/following-sibling::tr[1]//b[@class='stat-value chameleon-heading']/text()"
    )
    if len(lessons) == 1 and len(minutes) == 3:
        return {
            'lessons': int(lessons[0].strip()),
            'minutes': int(minutes[0].strip()) * 60 + int(minutes[1].strip())
        }

def parse_new_mail(content: str) -> dict | None:
    lessons = re.search(r'(\d+)\s*lessons?\s*\</h2', content.strip(), re.IGNORECASE)
    minutes = re.search(r'(\d+)\s*minutes?\s*\</h2', content.strip(), re.IGNORECASE)
    if lessons:
        return {
            'lessons': int(lessons.group(1)),
            'minutes': int(minutes.group(1))
        }        

parsers = [parse_old_mail, parse_new_mail]

In [3]:
directory = 'emails'
for filename in os.listdir(directory):
    if not filename.endswith('.eml'): 
        continue
    path = os.path.join(directory, filename)
    
    with open(path, 'r') as eml_file:
        msg = email.message_from_file(eml_file)

    if not 'duolingo' in msg['From'].casefold():
        continue

    print(msg['date'])
    for part in msg.walk():
        if part.get_content_type() != 'text/html':
            continue
        if part.get('Content-Transfer-Encoding') != 'base64':
            continue
        content = base64.b64decode(part.get_payload()).decode()

        result = next((result for parser in parsers if (result := parser(content))))
        print(result)

Sat, 5 Aug 2023 04:15:17 +0000
{'lessons': 11, 'minutes': 46}
Sat, 28 Dec 2024 17:27:55 +0000
{'lessons': 154, 'minutes': 497}
Sat, 2 Sep 2023 04:14:45 +0000
{'lessons': 14, 'minutes': 86}
